In [1]:
# customer segmentation using quantiles 
# reference: https://towardsdatascience.com/find-your-best-customers-with-customer-segmentation-in-python-61d602f9eee6
# data: http://archive.ics.uci.edu/ml/datasets/online+retail

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

In [4]:
# The dataset contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered online retailer.
# The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.
df = pd.read_excel("Online Retail.xlsx")

df1 = df # big dataset, so keep a copy before messing it up 

In [35]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# 1. EDA
Missing values in important columns;
Customers’ distribution in each country;
Unit price and Quantity should > 0;
Invoice date should < today.

In [5]:
df1.Country.nunique()
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [6]:
# The following function names data frame 1 (df1) "customer_country," and removes duplicates from each of the columns 'Country' and 'CustomerID'
customer_country = df1[['Country','CustomerID']].drop_duplicates()

# GROUPBY IS A VERY VERY USEFUL FUNCTION
# this formula organizes the data set first by country and then by customerID
# the aggregate and count aspect subtotals the data set by count of customerID by country
# the reset_index().sort values aspect ensures the data set will be arranged by highest to lowest total count of customerID per country (ascending order)
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)
# More than 90% of the customers in the data are from the United Kingdom.

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [7]:
# There’s some research indicating that customer clusters vary by geography, 
# so here I’ll restrict the data to the United Kingdom only.
# keep in mind that although we previously set different parameters for data frame 1 (DF1), 
# this function overwrites those parameters as we progress deeper into this exercise.
# specifically, in this case, "df1 = df1.loc" re-establishes data frame 1 as a location and identifies the United Kingdom as the country location 
# when you define a function, use the "=" sign. When you define a value, use the "==" sign.
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [8]:
# Check missing values
# Note that because of the function directly above that re-established data fram 1 (df1) 
# as the United Kingdom exclusively, the function below only identifies missing values specifically for the United Kingdom.
# In this function, "isnull" seeks the number of instances where there is nothing in the eight column fields in our data frame
# and the "sum" identifies the total number of instances in which there is no value.
df1.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [9]:
# There are 133,600 missing values in the CustomerID column and 1,454 missing values in the "Description" column for the United Kingdom, 
# and since our analysis is based on customers, we will remove these missing values.
# Once again, we are overwriting data frame 1 (df1). In most cases, a better habit might be to establish a new data frame (i.e., data frame 2)
# In overwriting data frame 1, we are re-establishing it as only idenitfying rows of data where a customerID is present 
# (i.e., "notnull," which means there's something as opposed to "isnull" which means there's nothing)
df1 = df1[pd.notnull(df1['CustomerID'])]

In [10]:
# Check the minimum value in the Quantity column.
df1.Quantity.min()

-80995

In [37]:
# Check the minimum value in the UnitPrice column.
df1.UnitPrice.min()

0.0

In [12]:
# Remove the negative values in the Quantity column.
# "shape" tells you the number of columns and rows 
# "info" tells you all available information applicable to the data frame such as "column" name, or "dtype" (data type)
df1 = df1[(df1['Quantity']>0)]
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    354345 non-null  object        
 1   StockCode    354345 non-null  object        
 2   Description  354345 non-null  object        
 3   Quantity     354345 non-null  int64         
 4   InvoiceDate  354345 non-null  datetime64[ns]
 5   UnitPrice    354345 non-null  float64       
 6   CustomerID   354345 non-null  float64       
 7   Country      354345 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [13]:
# This function identifies the total number of unique values for each column.---very useful
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [14]:
# Add a column for total price. 
# Specifically, we are creating the "Total Price" column by multiplying "Quantity" by "UnitPrice" as shown below.
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [15]:
# Find out the first order dates in the data.
df1['InvoiceDate'].min()


Timestamp('2010-12-01 08:26:00')

In [16]:
# Find out the last order dates in the data.
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [17]:
# Since recency is calculated for a point in time,
# and the last invoice date is 2011–12–09, we will use 2011–12–10 to calculate recency.
# "NOW" is a time we are defining using the function below.
# The pd.to_datetime (dt) method is used to convert the string datetime into a datetime object using pandas in python.
NOW = dt.datetime(2011,12,10)
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

# 2. RFM Customer Segmentation (recency, frequency, monetary_value)

## 2.1 Create a RFM table

In [22]:
# python is case sensitive
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()}).reset_index()
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [24]:
rfmTable.head()


,CustomerID,recency,frequency,monetary_value
0,12346.0,325,1,77183.60
1,12747.0,2,103,4196.01
2,12748.0,0,4596,33719.73
3,12749.0,3,199,4090.88
4,12820.0,3,59,942.34


In [25]:
# example interpretation:
first_customer = df1[df1['CustomerID'] == 12346.0]

first_customer
# CustomerID 12346 has frequency: 1, monetary value: $77,183.60 and recency: 325 days.

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6


In [27]:
# VS.RFM values
first_customer_rfm = rfmTable[rfmTable['CustomerID'] == 12346.0]
first_customer_rfm
# The first customer has shopped only once, bought one product at a huge quantity(74,215). 
# The unit price is very low; perhaps a clearance sale.

,CustomerID,recency,frequency,monetary_value
0,12346.0,325,1,77183.6


## 2.2 using quartiles to split customers into segments

In [28]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [29]:
### Create a segmented RFM table
segmented_rfm = rfmTable

In [30]:
# The lowest recency, highest frequency and monetary amounts are our best customers.
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [31]:
# segment numbers to the newly created segmented RFM table
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,CustomerID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
0,12346.0,325,1,77183.60,4,4,1
1,12747.0,2,103,4196.01,1,1,1
2,12748.0,0,4596,33719.73,1,1,1
3,12749.0,3,199,4090.88,1,1,1
4,12820.0,3,59,942.34,1,2,2


In [33]:
# RFM segments split the customer base into an imaginary 3D cube which is hard to visualize. 
# However, we can sort it out.

# Add a new column to combine RFM score: 111 is the highest score as we determined earlier.
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str)    + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()
# It is obvious that the first customer is not our best customer at all.

,CustomerID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,12346.0,325,1,77183.60,4,4,1,441
1,12747.0,2,103,4196.01,1,1,1,111
2,12748.0,0,4596,33719.73,1,1,1,111
3,12749.0,3,199,4090.88,1,1,1,111
4,12820.0,3,59,942.34,1,2,2,122


In [34]:
# find top 10 best customers
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,CustomerID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
3784,18102.0,0,431,259657.30,1,1,1,111
3315,17450.0,8,337,194550.79,1,1,1,111
3357,17511.0,2,963,91062.38,1,1,1,111
2767,16684.0,4,277,66653.56,1,1,1,111
903,14096.0,4,5111,65164.79,1,1,1,111
621,13694.0,3,568,65039.62,1,1,1,111
1772,15311.0,0,2379,60767.90,1,1,1,111
191,13089.0,2,1818,58825.83,1,1,1,111
2110,15769.0,7,130,56252.72,1,1,1,111
1580,15061.0,3,403,54534.14,1,1,1,111
